In [318]:
display_board()

 1234567 
+-------+
|.......|
|...X...|
|O.XOX..|
|X.OOOX.|
|X.XOOOO|
|XOXXOOX|
+-------+


In [14]:
test_dic = {}
test_score = 0

In [64]:
from IPython.display import clear_output
import random
import copy

# initiate board
EMPTY = '.'
board = []
for i in range(6):
    board.append([EMPTY]*7)
PLAYER = {True:'O',False:'X'}
current_player = True

display_board() # display board
while True:
    
    
    # get command location
    # if PLAYER[current_player] == ‘X’ -> then computer plays
    if current_player:
        location = input(f"Player {PLAYER[current_player]}'s move (1~7): ")
        # check if in 1-7
        if location.upper() == 'Q':
            print('Aborted')
            break
        if location not in [str(i) for i in range(1,8)]:
            print('Invalid move')
            continue
    else:
        location = AI_play()
        
        
    # check location viability and place the mark
    location = int(location)
    is_viable = get_viable(location)
    if is_viable is False:
        print('The location is already occupied')
        continue
    else:
        board[is_viable][location-1] = PLAYER[current_player]

    # check if anyone wins
    is_win = check_win()
    if is_win:
        clear_output()
        display_board()
        print(f"The winner is player '{PLAYER[current_player]}'!!")
        if PLAYER[current_player]=='X':
            print(f'The AI plays at {location}')
            print(advantage)
        break

    # check if the board is full
    is_full = check_full()
    if is_full:
        clear_output()
        display_board()
        print('A tie game.')
        break
    

    clear_output()  # clear output
    display_board() # display board
    
    # display computer
    if PLAYER[current_player]=='X':
        print(f'The AI plays at {location}')
        print('Pre', pre)
        print('Post', post)
        print(advantage)
        print(test_dic)
    
    # change player
    current_player = not current_player
    

 1234567 
+-------+
|.......|
|...X...|
|.OXO...|
|.OOOX..|
|.XXOO.X|
|.XOXOOX|
+-------+
The winner is player 'O'!!


In [18]:
floor = ['.','o']
floor.count('c')

0

In [56]:
def AI_play():
    # list all possible move
    possible_move = [i for i in range(1,8) if get_viable(i) is not False]  
    # 1 to 7 注意如果不用 is not False， 0 會被視為 false
    
    
    # calculate post-pre value, getting advatage    
    global advantage
    global pre
    global post
    advantage = {}
    pre = {}
    post = {}
    
    # count pre and post step to gain advantage
    for step in possible_move:
        pre_value = check_advantage()
        post_value = check_advantage(step)
        pre[step] = pre_value
        post[step] = post_value
        discount_next_O = check_O_next_step(step)  # discount for the best move of next move O
        advantage[step] = post_value - pre_value + discount_next_O  # record possible win-situation
        
        
    # calculate the advantages of each step
    # filter to the max value
    possible_move = [item for item in advantage.keys() if advantage[item]==max(advantage.values())]
    
    return str(random.choice(possible_move))  # return string form

def check_O_next_step(x_step):
    discount_score_for_each_O = []
    fake_board = copy.deepcopy(board)
    viable_row = get_viable(x_step)
    fake_board[viable_row][x_step-1]='X' # set 
    possible_move = [i for i in range(1,8) if get_viable_input(i,fake_board) is not False]  # 1-7
    
    # possible next O step
    for step in possible_move:
        discount_score_for_each_O.append(check_O_advantage(step, fake_board))
    
    test_dic[x_step] = discount_score_for_each_O
    
    return min(discount_score_for_each_O)  # return the minimal 
    
    
def check_O_advantage(y_step,bd):
    score = 0
    board_temp = copy.deepcopy(bd)
    viable_row = get_viable_input(y_step,board_temp) # get row num
    board_temp[viable_row][y_step-1]='O' # set O step

    
    # count score
    # if next step O will wing
    # check horizontal
    # last row
    
    floor = []
    
    for index, row in enumerate(board_temp):
        if index == len(board_temp)-1:  # last row
            for i in range(4):  # column 0,1,2,3
                if 'O'==row[i]==row[i+1]==row[i+2]==row[i+3]:
                    score = score - 100000
            for i in range(4):
                if 'O'==row[i+1]==row[i+2] and '.'==row[i]==row[i+3]:
                    score = score - 50000
        else:
            for i in range(4):  # column 0,1,2,3
                #floor = [board_temp[index-1][i],board_temp[index-1][i+1],board_temp[index-1][i+2],board_temp[index-1][i+3]]
                if 'O'==row[i]==row[i+1]==row[i+2]==row[i+3]:
                    score = score - 100000
            
    # check vertical
    for column in range(7):  
        for row_index in range(3):  # row 0,1,2
            if 'O'==board_temp[row_index][column]==board_temp[row_index+1][column]==board_temp[row_index+2][column]==board_temp[row_index+3][column]:
                score = score - 100000
    # check \
    for column in range(4):  # column 0,1,2,3
        for row_index in range(3):  # row 0,1,2
            #floor = [board_temp[row_index+1][column], board_temp[row_index+2][column+1], board_temp[row_index+3][column+2]]
            if 'O'==board_temp[row_index][column]==board_temp[row_index+1][column+1]==board_temp[row_index+2][column+2]==board_temp[row_index+3][column+3]:
                score = score - 100000
    # check /
    # column 3,2,1,0  row 0,1,2,3
    for column in range(4):  # column 0,1,2,3
        for row_index in range(3):  # row 0,1,2
            #floor = [board_temp[row_index+1][column+3], board_temp[row_index+2][column+2], board_temp[row_index+3][column+3]]
            if 'O'==board_temp[row_index][column+3]==board_temp[row_index+1][column+2]==board_temp[row_index+2][column+1]==board_temp[row_index+3][column]:
                score = score - 100000
    
    return score
    
def check_advantage(x_step=None):
    #x_step =x_step-1  # normalize to index
    score = 0
    
    # assign fake board
    fake_board = copy.deepcopy(board)
    
    # if step is givin, then is pre 
    if x_step is not None:
        viable_row = get_viable(x_step)  # get viable row
        fake_board[viable_row][x_step-1]='X'  # set
        score = score + viable_row * 10  # add score to bottom row
        
    # check for OOO pattern
    # check horizontal
    for row in board:
        for i in range(5):  # column 0,1,2,3
            if 'O'==row[i]==row[i+1]==row[i+2]:
                score = score - 10000
    # check vertical
    for column in range(7):  
        for row_index in range(4):  
            if 'O'==board[row_index][column]==board[row_index+1][column]==board[row_index+2][column]:
                score = score - 10000
    # check \
    for column in range(5):  # column 0,1,2,3
        for row_index in range(4):  # row 0,1,2
            if 'O'==board[row_index][column]==board[row_index+1][column+1]==board[row_index+2][column+2]:
                score = score - 10000
    # check /
    # column 3,2,1,0  row 0,1,2,3
    for column in range(5):  # column 0,1,2,3
        for row_index in range(4):  # row 0,1,2
            if 'O'==board[row_index][column+2]==board[row_index+1][column+1]==board[row_index+2][column]:
                score = score - 10000  
                
    # check row .O.O. pattern
    for row in board:
        for i in range(3):  # column 0,1,2,3
            if row[i]==row[i+2]==row[i+4]=='.' and 'O'==row[i+1]==row[i+3]:
                score = score - 5000
    # check row .OO. pattern
    for row in board:
        for i in range(4):  # column 0,1,2,3
            if row[i]==row[i+3]=='.' and 'O'==row[i+1]==row[i+2]:
                score = score - 5000
    
    # check row
    for row in fake_board:
        for i in range(4):  # column 0,1,2,3
            temp = [row[i],row[i+1],row[i+2],row[i+3]]
            O_is_exist = [item for item in temp if item=='O']
            X_is_exist = [item for item in temp if item=='X']
            if temp.count('O')==3 and temp.count('.')==1:
                score = score - 5000
            if O_is_exist:
                continue
            if len(X_is_exist)==4:
                score = score + 102400
            elif len(X_is_exist)==3:
                score = score + 512
            elif len(X_is_exist)==2:
                score = score + 10
            elif len(X_is_exist)==1:
                score = score + 1
    # check vertical
    for column in range(7):  
        for row_index in range(3):  # row 0,1,2
            temp = [fake_board[row_index][column],fake_board[row_index+1][column],fake_board[row_index+2][column],fake_board[row_index+3][column]]
            O_is_exist = [item for item in temp if item=='O']
            X_is_exist = [item for item in temp if item=='X']
            if temp.count('O')==3 and temp.count('.')==1:
                score = score - 5000
            if O_is_exist:
                continue
            elif len(X_is_exist)==4:
                score = score + 102400
            elif len(X_is_exist)==3:
                score = score + 512
            elif len(X_is_exist)==2:
                score = score + 10
            elif len(X_is_exist)==1:
                score = score + 1
            
    # check \
    for column in range(4):  # column 0,1,2,3
        for row_index in range(3):  # row 0,1,2
            temp = [fake_board[row_index][column],fake_board[row_index+1][column+1],fake_board[row_index+2][column+2],fake_board[row_index+3][column+3]]
            O_is_exist = [item for item in temp if item=='O']
            X_is_exist = [item for item in temp if item=='X']
            if temp.count('O')==3 and temp.count('.')==1:
                score = score - 5000
            if O_is_exist:
                continue
            elif len(X_is_exist)==4:
                score = score + 102400
            elif len(X_is_exist)==3:
                score = score + 512
            elif len(X_is_exist)==2:
                score = score + 10
            elif len(X_is_exist)==1:
                score = score + 1
    # check /
    # column 3,2,1,0  row 0,1,2,3
    for column in range(4):  # column 0,1,2,3
        for row_index in range(3):  # row 0,1,2
            temp = [fake_board[row_index][column+3],fake_board[row_index+1][column+2],fake_board[row_index+2][column+1],fake_board[row_index+3][column]]
            O_is_exist = [item for item in temp if item=='O']
            X_is_exist = [item for item in temp if item=='X']
            if temp.count('O')==3 and temp.count('.')==1:
                score = score - 5000
            if O_is_exist:
                continue
            elif len(X_is_exist)==4:
                score = score + 102400
            elif len(X_is_exist)==3:
                score = score + 512
            elif len(X_is_exist)==2:
                score = score + 10
            elif len(X_is_exist)==1:
                score = score + 1
    
    return score

In [58]:
def display_board():
    print(" 1234567 ")
    print("+-------+")
    for row in board:
        print('|',*row,'|',sep='')
    print("+-------+")
    
def get_viable(loc):
    global board
    bd = board
    # loc -> 1 to 7
    loc = int(loc)
    for blank_row in range(5,-1,-1): # count back 5,4,3,2,1,0
        if bd[blank_row][loc-1]==EMPTY:
            return blank_row
    return False

    
def get_viable_input(loc, bd):
    # loc -> 1 to 7
    loc = int(loc)
    for blank_row in range(5,-1,-1): # count back 5,4,3,2,1,0
        if bd[blank_row][loc-1]==EMPTY:
            return blank_row
    return False
    
def check_full():
    any_empty = any([item for row in board for item in row if item=='.' ])
    return not any_empty

def check_win():
    # check horizontal
    for row in board:
        for i in range(4):  # column 0,1,2,3
            if '.'!=row[i]==row[i+1]==row[i+2]==row[i+3]:
                return True
    # check vertical
    for column in range(7):  
        for row_index in range(3):  # row 0,1,2
            if '.'!=board[row_index][column]==board[row_index+1][column]==board[row_index+2][column]==board[row_index+3][column]:
                return True
    # check \
    for column in range(4):  # column 0,1,2,3
        for row_index in range(3):  # row 0,1,2
            if '.'!=board[row_index][column]==board[row_index+1][column+1]==board[row_index+2][column+2]==board[row_index+3][column+3]:
                return True
    # check /
    
    # column 3,2,1,0  row 0,1,2,3
    for column in range(4):  # column 0,1,2,3
        for row_index in range(3):  # row 0,1,2
            if '.'!=board[row_index][column+3]==board[row_index+1][column+2]==board[row_index+2][column+1]==board[row_index+3][column]:
                return True        
    
    return False